# Coastal change and impact analysis

* **Products used:** [DE Africa Coastlines](), [High Resolution Population Density]()


## Background

Coastal changes have a larget impact on human settlements. 

## Description

This notebook will demonstrate how to load relevant datasets and analyse the impact of coastal changes on human settlements.
Topics covered include:

1. Loading DE Africa Coastlines rates of change statistics data using the `get_coastlines` function.
2. Loading High Resolution Population Density.
3. Combine


## Getting started

To run this analysis, run all the cells in the notebook, starting with the "Load packages" cell.

### Load packages
Import Python packages that are used for the analysis.

In [1]:
import geopandas as gpd
import rioxarray as rxr
from deafrica_tools.spatial import xr_vectorize, xr_rasterize
from deafrica_tools.coastal import get_coastlines
from deafrica_tools.app import deacoastlines
from deafrica_tools.plotting import display_map

/usr/local/lib/python3.8/dist-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.10.3-CAPI-1.16.1). Conversions between both will be slow.
  warnings.warn(


### Analysis parameters

This section defines the analysis parameters, including:

* `central_lat, central_lon, buffer`: center lat/lon and analysis window size for the area of interest

The default location is an area on Nyali Beach along the Kenyan Coast.

In [21]:
# Set the central latitude and longitude.
#Niger
central_lat = 5
central_lon = 5
#Senegal
central_lat = 15
central_lon = -15

# Set the buffer to load around the central coordinates.
buffer = 5

# Compute the bounding box for the study area. 
xmin, xmax = central_lon - buffer, central_lon + buffer
ymin, ymax = central_lat - buffer, central_lat + buffer
bbox = (xmin, ymin, xmax, ymax)

## View the selected location

The next cell will display the selected area on an interactive map. Feel free to zoom in and out to get a better understanding of the area you'll be analysing. Clicking on any point of the map will reveal the latitude and longitude coordinates of that point.

In [22]:
display_map(x=(xmin, xmax), y=(ymin, ymax))

In [4]:
## Loading DE Africa Coastlines data 

In [32]:
#Lagos
#ds = rxr.open_rasterio('../../HighResolutionPopulationDensity/cog_globallat_0_lon_0_general-v1.5.7.tif', chunks ={'x': 1000, 'y': 1000})

#Senegal
ds = rxr.open_rasterio('../../HighResolutionPopulationDensity/cog_globallat_10_lon_-20_general-v1.5.3.tif', chunks ={'x': 1000, 'y': 1000})

#ds = rxr.open_rasterio('../../HighResolutionPopulationDensity/cog_globallat_0_lon_0_general-v1.5.7.tif', chunks ={'x': 1000, 'y': 1000})
#ds_youth = rxr.open_rasterio('../../HighResolutionPopulationDensity/cog_globallat_0_lon_0_youth_15_24-v1.5.7.tif', chunks ={'x': 1000, 'y': 1000})

### Population within a buffer of 2021 shoreline

In [33]:
buffer = 50 #meter

In [ ]:
#2021 coastlines

# Load DE Africa Coastlines annual shorelines data. 
deacl_shorelines_gdf = get_coastlines(bbox=bbox, 
                                      layer='shorelines')

In [ ]:
buffered = deacl_shorelines_gdf.copy()

In [ ]:
buffered['geometry'] = deacl_shorelines_gdf.to_crs('6933').buffer(buffer).to_crs('4326')

In [ ]:
coastal_bbox = buffered.unary_union.bounds
coastal_bbox

In [ ]:
pop_raster = ds.isel(band=0).sel(x=slice(coastal_bbox[0],coastal_bbox[2]), y = slice(coastal_bbox[3],coastal_bbox[1]))#.compute()
#youth_raster = ds_youth.isel(band=0).sel(x=slice(coastal_bbox[0],coastal_bbox[2]), y = slice(coastal_bbox[3],coastal_bbox[1]))

In [ ]:
pop_raster.geobox

In [ ]:
pop_raster=pop_raster.rename({'x':'longitude','y':'latitude'})
#youth_raster=youth_raster.rename({'x':'longitude','y':'latitude'})

In [ ]:
coastal_pop = {}
for year in range(2000,2021):
    coastal_area = buffered[buffered.year==str(year)]
    coastal_raster = xr_rasterize(gdf=coastal_area,
                                  da=pop_raster,
                                  transform=pop_raster.geobox.transform,
                                  crs=pop_raster.geobox.crs)
    
    #fraction of coastal population
    coastal_pop[year] = (coastal_raster*pop_raster).sum().values

In [ ]:
import pandas as pd
import numpy as np

# convert to million
pd.DataFrame(np.array(list(coastal_pop.values()))/1e6, 
             columns=[f'General'], 
             index=[str(x) for x in coastal_pop.keys()]
            ).plot(ylabel=f'Population within {buffer}m from coast (million)',xlabel='Year')